- country: país onde os dados foram registrados (101 países)
- year: ano em que os dados foram registrados (1985–2016)
- sex: sexo considerado no registro
- male - masculino
- female - feminino
- age: faixa etária considerada
- 5–14 anos
- 15–24 anos
- 25–34 anos
- 35–54 anos
- 55–74 anos
- 75+ anos
- suicides_no: número de suicídios
- population: população para o grupo
- suicides/100k pop: número de suicídios por 100 mil habitantes
- country_year: identificador contendo country + year
- HDI for year: Índice de Desenvolvimento Humano (IDH) para o ano
- gdp_for_year: Produto Interno Bruto (PIB) para o ano
- gdp_per_capita: Produto Interno Bruto (PIB) per capita

In [55]:

import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

import seaborn as sns
sns.set(style = 'dark')

In [56]:
df = pd.read_csv('./data/master.csv')
df['sex']=df['sex'].apply(lambda x: 'masculino' if x == 'male' else ('feminino' if x == 'female' else ''))
df['age']=df['age'].apply(lambda x: x[0:-5] + 'anos' if x[-5:] == 'years' else x)

df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,masculino,15-24 anos,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,masculino,35-54 anos,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,feminino,15-24 anos,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,masculino,75+ anos,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,masculino,25-34 anos,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [57]:
# Gráfico IDH menor - Rian
idh_menor = 0.60

dados_menor = df[df['HDI for year'] < idh_menor]

dados_agrupados_menor = dados_menor.groupby(['country', 'age'])['suicides/100k pop'].mean().reset_index()

fig_menor = px.bar(dados_agrupados_menor, x='country', y='suicides/100k pop', color='age', text='suicides/100k pop',
             title='Suicídios a cada 100 mil pessoas em Países com IDH Baixo (por faixa etária)',
             labels={'suicides/100k pop': 'Suicídios a cada 100 mil pessoas', 'country': 'País', 'age': 'Faixa Etária'},
             category_orders={'age': ['5-14 anos', '15-24 anos', '25-34 anos', '35-54 anos', '55-74 anos', '75+ anos']},
             height=600)

fig_menor.update_traces(texttemplate='%{text:.2f}', textposition='outside', marker=dict(line=dict(color='black', width=1)))

fig_menor.update_layout(barmode='group', xaxis_title='', yaxis_title='Suicídios a cada 100 mil pessoas',
                  legend_title='Faixas Etárias', title_x=0.5, title_font=dict(size=20),
                  plot_bgcolor='white', showlegend=True)

fig_menor.write_html('./charts/idh_menor.html', auto_open=False)

In [58]:
# Gráfico IDH maior - Rian
idh_maior = 0.90

dados_maior = df[df['HDI for year'] > idh_maior]

dados_agrupados_maior = dados_maior.groupby(['country', 'age'])['suicides/100k pop'].mean().reset_index()

fig_maior = px.bar(dados_agrupados_maior, x='country', y='suicides/100k pop', color='age', text='suicides/100k pop',
             title='Suicídios a cada 100 mil pessoas em Países com IDH Alto (por faixa etária)',
             labels={'suicides/100k pop': 'Suicídios a cada 100 mil pessoas', 'country': 'País', 'age': 'Faixa Etária'},
             category_orders={'age': ['5-14 anos', '15-24 anos', '25-34 anos', '35-54 anos', '55-74 anos', '75+ anos']},
             height=600)

fig_maior.update_traces(texttemplate='%{text:.2f}', textposition='outside', marker=dict(line=dict(color='black', width=1)))
fig_maior.update_layout(barmode='group', xaxis_title='', yaxis_title='Suicídios a cada 100 mil pessoas',
                  legend_title='Faixas Etárias', title_x=0.5, title_font=dict(size=20),
                  plot_bgcolor='white', showlegend=True)

fig_maior.write_html('./charts/idh_maior.html', auto_open=False)

In [59]:
# Indice por faixa etária - Natã
df_brasil = df[df.country == 'Brazil'].copy()
df_filtered = df_brasil[df_brasil['sex'].isin(['masculino', 'feminino'])]

# Agrupar por idade e sexo
idade = df_filtered.groupby(['age', 'sex'])['suicides_no'].sum().reset_index()

# Ordenar as faixas etárias
ordem_faixa_etaria = ['5-14 anos', '15-24 anos', '25-34 anos', '35-54 anos', '55-74 anos', '75+ anos']
idade['age'] = pd.Categorical(idade['age'], categories=ordem_faixa_etaria, ordered=True)

# Criar o gráfico de barras empilhadas
fig = px.bar(idade, x='age', y='suicides_no', color='sex',
             title='Total de Suicídios por Faixa Etária e Gênero (1985 - 2016)',
             labels={'age': 'Faixa Etária', 'suicides_no': 'Número de Suicídios', 'sex': 'Gênero'},
             barmode='group')  # Usar barmode 'group' para barras lado a lado

fig.update_xaxes(categoryorder='array', categoryarray=ordem_faixa_etaria)

# Personalize a estética do gráfico
fig.update_traces(marker_line_width=0)  # Remova linhas das barras
fig.update_layout(
    showlegend=True,
    legend_title_text='Gênero',  # Título da legenda
    xaxis_title='Faixa Etária',  # Título do eixo X
    yaxis_title='Número de Suicídios',  # Título do eixo Y
    margin=dict(l=50, r=50, b=100, t=100),  # Margens do gráfico
    template='plotly',  # Esquema de cores
)

# Adicione rótulos de dados nas barras
fig.update_traces(texttemplate='%{y}', textposition='outside')

# Salvar o gráfico como arquivo HTML
fig.write_html('./charts/faixa_etária.html', auto_open=False)


In [60]:
# Suicidios por genero - Natã

df_brasil = df[df.country == 'Brazil'].copy()

genero = df_brasil.groupby('sex')['suicides_no'].sum().reset_index()

# Criar o gráfico de pizza aprimorado
fig = px.pie(genero, values='suicides_no', names='sex', title='Distribuição de Suicídios por Gênero',
             color_discrete_sequence=['#ff6f61', '#55a630'], hole=0.3, height=400, width=600)

# Exibir a porcentagem e o rótulo dentro de cada fatia
fig.update_traces(textinfo='percent+label', textposition='inside', pull=[0.05, 0.05])

# Personalizar a legenda
fig.update_layout(
    legend=dict(title='Gênero', x=0.7, y=0.5),
)

# Personalizar o rótulo ao passar o cursor
fig.update_traces(hovertemplate='%{label}: %{value} (suicídios)')

# Adicionar um subtítulo ou notas explicativas
subtitulo = "Gráfico de pizza que mostra a distribuição de suicídios por gênero no Brasil."
fig.add_annotation(text=subtitulo, xref="paper", yref="paper", x=0.5, y=-0.2, showarrow=False)

# Salvar o gráfico como arquivo HTML
fig.write_html('./charts/genero.html', auto_open=True)

In [ ]:
# Indice de suicidio por genero do Brasil - Vitor
df_brasil = df[df.country == 'Brazil'].copy()
anos = df_brasil.groupby(['year', 'sex'])['suicides_no'].sum().reset_index()

# Filtrar dados por gênero
dados_masculinos = anos[anos['sex'] == 'masculino']
dados_femininos = anos[anos['sex'] == 'feminino']

# Criar traces para gráficos de barras
trace_masculino = go.Bar(
    x=dados_masculinos['year'],
    y=dados_masculinos['suicides_no'],
    name='Masculino',
    marker=dict(color='blue'),
    hovertemplate='Ano: %{x}<br>Gênero: Masculino<br>Número de suicídios: %{y}<extra></extra>'
)

trace_feminino = go.Bar(
    x=dados_femininos['year'],
    y=dados_femininos['suicides_no'],
    name='Feminino',
    marker=dict(color='violet'),
    hovertemplate='Ano: %{x}<br>Gênero: Feminino<br>Número de suicídios: %{y}<extra></extra>'
)

# Layout do gráfico
layout = go.Layout(
    title='Número de Suicídios por Gênero no Brasil',
    xaxis=dict(title='Anos'),
    yaxis=dict(title='Número de suicídios'),
    barmode='group',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#f2f2f2',
    font=dict(color='#333333')
)

# Criar figura e adicionar traces
fig = go.Figure(data=[trace_masculino, trace_feminino], layout=layout)

# Salvar o gráfico em um arquivo HTML
fig.write_html('./charts/indice_suicidio_genero_brasil.html', auto_open=False)

In [ ]:
# Brasil x Mundo - Vitor
brazil_tb = df[df['country'] == 'Brazil']
brazil_tb = brazil_tb.groupby('year').apply(lambda x: round((x['suicides_no'].sum() / x['population'].sum()) * 100000, 2)).reset_index(name='Brasil')

world_tb = df.groupby('year').apply(lambda x: round((x['suicides_no'].sum() / x['population'].sum()) * 100000, 2)).reset_index(name='Mundo')

result = pd.merge(world_tb, brazil_tb, on='year')

fig = px.bar(result, x='year', y=['Mundo', 'Brasil'],
             barmode='group', color_discrete_map={'Mundo': '#3498db', 'Brasil': '#e74c3c'},
             labels={'value': 'Número de Suicídios', 'year': 'Ano', 'variable': 'Local'},
             title='Comparação do índice de suicídio (Brasil x Mundo)',
             )

fig.update_traces(marker=dict(line=dict(color='white', width=1)))

fig.update_xaxes(title_text='Ano', showgrid=False)
fig.update_yaxes(title_text='Numero de Suicídios por 100 mil habitantes', showgrid=True, gridcolor='lightgray')

fig.update_layout(legend_title_text='Local')

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='lightgray',
    font=dict(color='black'),
)

fig.update_layout(title=dict(x=0.5, y=0.95, xanchor='center', yanchor='top'))

fig.update_layout(margin=dict(b=50))

fig.write_html('./charts/comparacao_suicidios.html', auto_open=False)